In [54]:
import pandas as pd
import numpy as np
from gurobipy import GRB
import gurobipy as gp

In [55]:
demand = [0,0,0,5000,6000,6500,7000,8000,9500]

In [56]:
m = gp.Model("Production")

In [57]:
## Allocate decision variables
x = m.addMVar(shape=(len(demand)), vtype=GRB.CONTINUOUS , name="decision")

In [58]:
## Add constraints on max/min production values
m.addConstr(x[0] == 500)
m.addConstr(x[1] == 2000)
m.addConstr(x[2] == 1000)

for i in range(3, x.shape[0]):
    m.addConstr(
        0 <= x[i]
    )
    m.addConstr(
        x[i] <= 8000
    )

In [59]:
## Add intermediate variable y to represent stock allocations
## y[i][0] -> production for current month, y[i][1] -> residual from month i-1, y[i][2] -> residual from month i-2
y = m.addMVar(shape=(len(demand), 3), vtype=GRB.CONTINUOUS , name="alloc")

for i in range(x.shape[0]):
    for j in range(3):
        m.addConstr(
            y[i][j] >= 0
            )

In [60]:
## Add basic constraints on allocations
for i in range(x.shape[0]):
    m.addConstr(
        x[i] == gp.quicksum(y[i][j] for j in range(3))
    )

In [61]:
## Add constracts to meet demand of each month
for i in range(3, x.shape[0]):
    m.addConstr(
        demand[i] <= y[i][0] + y[i-1][1]*0.89 + y[i-2][2]*0.47
    )

In [62]:
obj_f = sum([((x[i]*15) + (y[i][1]*0.11)*25 + (y[i][2]*0.53)*25 + (y[i][1])*0.75 + (y[i][2])*0.75*2) for i in range(x.shape[0])])

In [63]:
m.setObjective(
    obj_f
    ,GRB.MINIMIZE
)

In [64]:
try: 
    m.optimize()
except gp.GurobiError as E:
    print("Optimize failed", E)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 57 rows, 36 columns and 96 nonzeros
Model fingerprint: 0x5b790da4
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [4e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 1e+04]
Presolve removed 47 rows and 15 columns
Presolve time: 0.03s
Presolved: 10 rows, 21 columns, 35 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6549730e+05   5.710861e+03   0.000000e+00      0s
       9    6.9625374e+05   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.04 seconds (0.00 work units)
Optimal objective  6.962537436e+05


In [65]:
for i in range(x.shape[0]):
    print(x[i].x, y[i].x)

500.0 [500.   0.   0.]
2000.0 [2000.    0.    0.]
1000.0 [   0. 1000.    0.]
4110.0 [4110.    0.    0.]
6000.0 [6000.    0.    0.]
7504.1576296262665 [6500.         1004.15762963    0.        ]
8000.0 [6106.29970963 1893.70029037    0.        ]
8000.0 [6314.60674157 1685.39325843    0.        ]
8000.0 [8000.    0.    0.]


In [66]:
for i in range(x.shape[0]):
    print("Month %d: %d %.2f %.2f %.2f"%(i, x[i].x, *y[i].x))

Month 0: 500 500.00 0.00 0.00
Month 1: 2000 2000.00 0.00 0.00
Month 2: 1000 0.00 1000.00 0.00
Month 3: 4110 4110.00 0.00 0.00
Month 4: 6000 6000.00 0.00 0.00
Month 5: 7504 6500.00 1004.16 0.00
Month 6: 8000 6106.30 1893.70 0.00
Month 7: 8000 6314.61 1685.39 0.00
Month 8: 8000 8000.00 0.00 0.00


In [67]:
sum([((y[i][1].x*0.11)*25 + (y[i][2].x*0.53)*25 + (y[i][1].x)*0.75 + (y[i][2].x)*0.75*2) for i in range(x.shape[0])])

19541.379124472136

In [68]:
1005 * 0.89 + 6106

7000.45